<a href="https://colab.research.google.com/github/maxla777/Project-2/blob/Monique/Monique/ML_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fundamental Analysis

Market cap = current price * circulating supply
1.    NVT Ratio: (Network Value (market cap)) / Daily USD volume
    * NVT indicates whether a coin's price is overvalued or undervalued
    * NVT is a good indicator for revealing speculative value and possible bubbles
2.    MVRV Ratio: (Market cap) / (Realized cap)
    *   Good instrument for estimating if market participants are in profit or not
    *   MVRV is an excellent tool for determining if bitcoin is overvalued or undervalued during market cycles
    *   Negative MVRV ratio indicates market participants being not in profit, or minimally in profit 
    *   Positive MVRV indicates asset holders likely being in profit



In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
csv_path = Path('/content/drive/MyDrive/Colab Notebooks/btc.csv')
btc_data=pd.read_csv(csv_path)
btc_data.tail()

,time,AdrActCnt,BlkCnt,BlkSizeByte,BlkSizeMeanByte,CapMVRVCur,CapMrktCurUSD,CapRealUSD,DiffMean,FeeMeanNtv,FeeMeanUSD,FeeMedNtv,FeeMedUSD,FeeTotNtv,FeeTotUSD,HashRate,IssContNtv,IssContPctAnn,IssContUSD,IssTotNtv,IssTotUSD,NVTAdj,NVTAdj90,PriceBTC,PriceUSD,ROI1yr,ROI30d,SplyCur,SplyExpFut10yrCMBI,SplyFF,TxCnt,TxTfrCnt,TxTfrValAdjNtv,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,TxTfrValNtv,TxTfrValUSD,VtyDayRet180d,VtyDayRet30d,VtyDayRet60d
4448,2021-03-09T00:00:00.000Z,1088598.0,140.0,183831817.0,1.313084e+06,3.498556,1.020849e+12,2.917916e+11,2.144828e+13,0.000355,19.423635,0.000167,9.167163,106.248179,5.815933e+06,1.492680e+08,875.00,1.712527,4.789674e+07,875.00,4.789674e+07,66.214852,105.619998,1.0,54739.135254,593.217631,40.345917,1.864934e+07,2.057739e+07,1.455747e+07,299423.0,938346.0,281648.968287,1.541722e+10,0.616635,33754.086642,0.003600,197.060887,578617.328109,3.167301e+10,0.038893,0.051790,0.051605
4449,2021-03-10T00:00:00.000Z,1165773.0,149.0,194008352.0,1.302069e+06,3.562933,1.046580e+12,2.937410e+11,2.144828e+13,0.000319,17.907751,0.000170,9.511669,109.350972,6.136344e+06,1.588638e+08,931.25,1.822445,5.225806e+07,931.25,5.225806e+07,88.596092,107.409868,1.0,56116.040912,609.438086,21.668881,1.865028e+07,2.057762e+07,1.455754e+07,342661.0,982213.0,210509.016138,1.181293e+10,0.498878,27995.045791,0.003522,197.632840,490004.238791,2.749710e+10,0.038909,0.042687,0.051630
4450,2021-03-11T00:00:00.000Z,1084417.0,132.0,168874631.0,1.279353e+06,3.631770,1.078915e+12,2.970769e+11,2.144828e+13,0.000382,22.124840,0.000198,11.480945,112.244756,6.493051e+06,1.407384e+08,825.00,1.614516,4.772399e+07,825.00,4.772399e+07,79.352321,109.556390,1.0,57847.254936,628.615291,24.273983,1.865110e+07,2.057783e+07,1.456772e+07,293471.0,922585.0,235041.657001,1.359651e+10,0.588608,34049.338230,0.003529,204.123295,543040.611791,3.141341e+10,0.038939,0.042907,0.051171
4451,2021-03-12T00:00:00.000Z,1186618.0,150.0,197235362.0,1.314902e+06,3.570586,1.069408e+12,2.995049e+11,2.144828e+13,0.000364,20.854329,0.000177,10.145938,116.419908,6.674894e+06,1.599300e+08,937.50,1.834585,5.375123e+07,937.50,5.375123e+07,80.492506,107.390112,1.0,57334.641535,1056.100386,27.189490,1.865204e+07,2.057806e+07,1.456883e+07,320069.0,983349.0,231723.915787,1.328581e+10,0.537480,30816.215737,0.002970,170.283885,528530.293689,3.030309e+10,0.038933,0.042380,0.049837
4452,2021-03-13T00:00:00.000Z,1104466.0,143.0,201509838.0,1.409160e+06,3.789576,1.143216e+12,3.016738e+11,2.144828e+13,0.000421,25.808098,0.000194,11.913316,116.326589,7.129516e+06,1.524666e+08,893.75,1.748715,5.477686e+07,893.75,5.477686e+07,99.633855,113.971254,1.0,61288.794128,989.057288,27.970680,1.865293e+07,2.057828e+07,1.457003e+07,276253.0,870843.0,187214.802059,1.147417e+10,0.520009,31870.733623,0.002990,183.275558,452846.326564,2.775441e+10,0.039129,0.042655,0.049903


In [2]:
btc_data.set_index(pd.to_datetime(btc_data['time'], infer_datetime_format=True), inplace=True)
btc_data.drop(columns=['time'], inplace=True)
btc_data.tail()


,AdrActCnt,BlkCnt,BlkSizeByte,BlkSizeMeanByte,CapMVRVCur,CapMrktCurUSD,CapRealUSD,DiffMean,FeeMeanNtv,FeeMeanUSD,FeeMedNtv,FeeMedUSD,FeeTotNtv,FeeTotUSD,HashRate,IssContNtv,IssContPctAnn,IssContUSD,IssTotNtv,IssTotUSD,NVTAdj,NVTAdj90,PriceBTC,PriceUSD,ROI1yr,ROI30d,SplyCur,SplyExpFut10yrCMBI,SplyFF,TxCnt,TxTfrCnt,TxTfrValAdjNtv,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,TxTfrValNtv,TxTfrValUSD,VtyDayRet180d,VtyDayRet30d,VtyDayRet60d
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-03-09 00:00:00+00:00,1088598.0,140.0,183831817.0,1.313084e+06,3.498556,1.020849e+12,2.917916e+11,2.144828e+13,0.000355,19.423635,0.000167,9.167163,106.248179,5.815933e+06,1.492680e+08,875.00,1.712527,4.789674e+07,875.00,4.789674e+07,66.214852,105.619998,1.0,54739.135254,593.217631,40.345917,1.864934e+07,2.057739e+07,1.455747e+07,299423.0,938346.0,281648.968287,1.541722e+10,0.616635,33754.086642,0.003600,197.060887,578617.328109,3.167301e+10,0.038893,0.051790,0.051605
2021-03-10 00:00:00+00:00,1165773.0,149.0,194008352.0,1.302069e+06,3.562933,1.046580e+12,2.937410e+11,2.144828e+13,0.000319,17.907751,0.000170,9.511669,109.350972,6.136344e+06,1.588638e+08,931.25,1.822445,5.225806e+07,931.25,5.225806e+07,88.596092,107.409868,1.0,56116.040912,609.438086,21.668881,1.865028e+07,2.057762e+07,1.455754e+07,342661.0,982213.0,210509.016138,1.181293e+10,0.498878,27995.045791,0.003522,197.632840,490004.238791,2.749710e+10,0.038909,0.042687,0.051630
2021-03-11 00:00:00+00:00,1084417.0,132.0,168874631.0,1.279353e+06,3.631770,1.078915e+12,2.970769e+11,2.144828e+13,0.000382,22.124840,0.000198,11.480945,112.244756,6.493051e+06,1.407384e+08,825.00,1.614516,4.772399e+07,825.00,4.772399e+07,79.352321,109.556390,1.0,57847.254936,628.615291,24.273983,1.865110e+07,2.057783e+07,1.456772e+07,293471.0,922585.0,235041.657001,1.359651e+10,0.588608,34049.338230,0.003529,204.123295,543040.611791,3.141341e+10,0.038939,0.042907,0.051171
2021-03-12 00:00:00+00:00,1186618.0,150.0,197235362.0,1.314902e+06,3.570586,1.069408e+12,2.995049e+11,2.144828e+13,0.000364,20.854329,0.000177,10.145938,116.419908,6.674894e+06,1.599300e+08,937.50,1.834585,5.375123e+07,937.50,5.375123e+07,80.492506,107.390112,1.0,57334.641535,1056.100386,27.189490,1.865204e+07,2.057806e+07,1.456883e+07,320069.0,983349.0,231723.915787,1.328581e+10,0.537480,30816.215737,0.002970,170.283885,528530.293689,3.030309e+10,0.038933,0.042380,0.049837
2021-03-13 00:00:00+00:00,1104466.0,143.0,201509838.0,1.409160e+06,3.789576,1.143216e+12,3.016738e+11,2.144828e+13,0.000421,25.808098,0.000194,11.913316,116.326589,7.129516e+06,1.524666e+08,893.75,1.748715,5.477686e+07,893.75,5.477686e+07,99.633855,113.971254,1.0,61288.794128,989.057288,27.970680,1.865293e+07,2.057828e+07,1.457003e+07,276253.0,870843.0,187214.802059,1.147417e+10,0.520009,31870.733623,0.002990,183.275558,452846.326564,2.775441e+10,0.039129,0.042655,0.049903


In [3]:
btc_data = btc_data.dropna().copy()
btc_data.head()

,AdrActCnt,BlkCnt,BlkSizeByte,BlkSizeMeanByte,CapMVRVCur,CapMrktCurUSD,CapRealUSD,DiffMean,FeeMeanNtv,FeeMeanUSD,FeeMedNtv,FeeMedUSD,FeeTotNtv,FeeTotUSD,HashRate,IssContNtv,IssContPctAnn,IssContUSD,IssTotNtv,IssTotUSD,NVTAdj,NVTAdj90,PriceBTC,PriceUSD,ROI1yr,ROI30d,SplyCur,SplyExpFut10yrCMBI,SplyFF,TxCnt,TxTfrCnt,TxTfrValAdjNtv,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,TxTfrValNtv,TxTfrValUSD,VtyDayRet180d,VtyDayRet30d,VtyDayRet60d
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2011-07-18 00:00:00+00:00,27778.0,170.0,4578379.0,26931.641176,1.942074,9.334931e+07,4.806681e+07,1.563028e+06,0.001486,0.020256,0.0000,0.000000,12.946993,176.474270,13.208752,8500.0,45.301560,115859.430629,8500.0,115859.430629,19.892789,30.989318,1.0,13.630521,15778.985613,-19.228477,6848549.99,18578587.50,6.848547e+06,8712.0,23705.0,344272.985389,4.692620e+06,139.134619,1896.477386,0.35000,4.770682,3.298186e+06,4.495600e+07,0.096840,0.039919,0.115691
2011-07-19 00:00:00+00:00,25356.0,142.0,4178498.0,29426.042254,1.964390,9.477795e+07,4.824804e+07,1.588241e+06,0.001642,0.022704,0.0000,0.000000,14.831258,205.039097,11.211171,7100.0,37.800938,98156.043355,7100.0,98156.043355,34.786764,31.172344,1.0,13.824795,17009.894602,-21.049521,6855649.99,18579475.00,6.855647e+06,9031.0,24777.0,197076.395947,2.724541e+06,110.410539,1526.403051,0.31234,4.318042,2.735642e+06,3.781969e+07,0.095685,0.039263,0.112180
2011-07-20 00:00:00+00:00,25461.0,149.0,3834661.0,25735.979866,1.962968,9.387575e+07,4.782336e+07,1.690896e+06,0.001202,0.016445,0.0000,0.000000,10.438026,142.774767,12.524178,7450.0,39.621308,101903.562911,7450.0,101903.562911,27.946156,30.515647,1.0,13.678331,18202.264271,-21.885947,6863099.99,18580406.25,6.863097e+06,8682.0,24251.0,245582.970111,3.359165e+06,22.315422,305.237722,0.20000,2.735666,5.411713e+05,7.402320e+06,0.095626,0.039237,0.111902
2011-07-21 00:00:00+00:00,25378.0,152.0,3672287.0,24159.782895,1.952017,9.352265e+07,4.791077e+07,1.690896e+06,0.001113,0.015150,0.0000,0.000000,8.860440,120.606610,12.776343,7600.0,40.374343,103449.744158,7600.0,103449.744158,49.877146,30.215741,1.0,13.611808,17088.175785,-22.265841,6870699.99,18581356.25,6.870697e+06,7961.0,24592.0,137752.469191,1.875060e+06,54.063834,735.906549,0.13000,1.769535,1.329538e+06,1.809741e+07,0.095621,0.039212,0.111371
2011-07-22 00:00:00+00:00,24428.0,151.0,3850125.0,25497.516556,1.963113,9.422203e+07,4.799623e+07,1.690896e+06,0.001555,0.021301,0.0005,0.006849,13.870556,190.006464,12.692288,7550.0,40.064697,103424.032350,7550.0,103424.032350,43.467176,30.232118,1.0,13.698547,23328.430771,-21.770493,6878249.99,18582300.00,6.878247e+06,8920.0,23839.0,158240.094166,2.167659e+06,65.213669,893.332526,0.50000,6.849274,1.554629e+06,2.129615e+07,0.095620,0.039276,0.111251


In [4]:
# Copy needed columns - CapMrktCurUSD & CapRealUSD
btc_cap_df = btc_data[['CapMrktCurUSD', 'CapRealUSD']].copy()
btc_cap_df.tail()

,CapMrktCurUSD,CapRealUSD
time,,
2021-03-09 00:00:00+00:00,1.020849e+12,2.917916e+11
2021-03-10 00:00:00+00:00,1.046580e+12,2.937410e+11
2021-03-11 00:00:00+00:00,1.078915e+12,2.970769e+11
2021-03-12 00:00:00+00:00,1.069408e+12,2.995049e+11
2021-03-13 00:00:00+00:00,1.143216e+12,3.016738e+11


In [5]:
btc_cap_df['MVRV'] = btc_cap_df['CapMrktCurUSD'] / btc_cap_df['CapRealUSD']
btc_cap_df.tail()

,CapMrktCurUSD,CapRealUSD,MVRV
time,,,
2021-03-09 00:00:00+00:00,1.020849e+12,2.917916e+11,3.498556
2021-03-10 00:00:00+00:00,1.046580e+12,2.937410e+11,3.562933
2021-03-11 00:00:00+00:00,1.078915e+12,2.970769e+11,3.631770
2021-03-12 00:00:00+00:00,1.069408e+12,2.995049e+11,3.570586
2021-03-13 00:00:00+00:00,1.143216e+12,3.016738e+11,3.789576


In [6]:
btc_cap_df['MVRV_change'] = btc_cap_df['MVRV'].diff()
btc_cap_df.tail()

,CapMrktCurUSD,CapRealUSD,MVRV,MVRV_change
time,,,,
2021-03-09 00:00:00+00:00,1.020849e+12,2.917916e+11,3.498556,0.148533
2021-03-10 00:00:00+00:00,1.046580e+12,2.937410e+11,3.562933,0.064378
2021-03-11 00:00:00+00:00,1.078915e+12,2.970769e+11,3.631770,0.068837
2021-03-12 00:00:00+00:00,1.069408e+12,2.995049e+11,3.570586,-0.061184
2021-03-13 00:00:00+00:00,1.143216e+12,3.016738e+11,3.789576,0.218990


# Machine Learning Models

AdaBoost

In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

In [8]:
# Usage from scikit learn documentation
# To update to fit our project
# from sklearn.model_selection import cross_val_score
# from sklearn.datasets import load_iris
# from sklearn.ensemble import AdaBoostClassifier

# X, y = load_iris(return_X_y=True)
# clf = AdaBoostClassifier(n_estimators=100)
# scores = cross_val_score(clf, X, y, cv=5)
# scores.mean()

Random Forest

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [10]:
# Fit a SKLearn linear regression using just the training set (X_train, y_train):
# model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)
# model.fit(X_train, y_train)

# Make a prediction of "y" values from the X_test dataset
# predictions = model.predict(X_test)

# Assemble actual y data (Y_test) with predicted y data (from just above) into two columns in a dataframe:
# Results = y_test.to_frame()
# Results["Predicted Value"] = predictions
# Results